In [1]:
import subprocess
import os
import numpy as np
import pandas as pd

import matplotlib

from datetime import datetime

from sklearn import linear_model

from scipy import signal


pd.set_option('display.max_rows', 500)

![CRSIP_DM](..\reports\figures\CRISP_DM.png)

# Data Preparation

* focus on understanding the final data structure
* support each step by visual analytics

## John Hopkins github CSV data

In [2]:
git_pull = subprocess.Popen( "git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   a10636a9..4c8006b0  master               -> origin/master\n * [new branch]        patch-08-18-us-daily -> origin/patch-08-18-us-daily\n   6854f5ab..ff9f793b  web-data             -> origin/web-data\n'
out : b'Updating a10636a9..4c8006b0\nFast-forward\n README.md                                          |    7 +\n csse_covid_19_data/README.md                       |   28 +-\n csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv   |    2 +-\n .../csse_covid_19_daily_reports/08-18-2020.csv     | 7896 ++++++++++----------\n .../csse_covid_19_daily_reports/08-24-2020.csv     | 3951 ++++++++++\n .../csse_covid_19_daily_reports/08-25-2020.csv     | 3951 ++++++++++\n .../csse_covid_19_daily_reports/08-26-2020.csv     | 3951 ++++++++++\n .../csse_covid_19_daily_reports/08-27-2020.csv     | 3951 ++++++++++\n .../csse_covid_19_daily_reports/08-28-2020.csv     | 3951 ++++++++++\n .../csse_covid_19_daily_reports/08-29-2020.csv     | 3951 +++

In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38243,38288,38304,38324,38398,38494,38520,38544,38572,38606
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9728,9844,9967,10102,10255,10406,10553,10704,10860,11021
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,45158,45469,45773,46071,46364,46653,46938,47216,47488,47752
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1199,1199,1215,1215,1215,1261,1261,1301,1301,1344
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2777,2805,2876,2935,2965,2981,3033,3092,3217,3279


In [5]:
time_index = pd_raw.columns[4:]

In [6]:
df_plot = pd.DataFrame({
    'date': time_index
})
df_plot.head()

,date
0,1/22/20
1,1/23/20
2,1/24/20
3,1/25/20
4,1/26/20


In [7]:
type(df_plot['date'][0])

str

In [8]:
country_set = pd_raw['Country/Region'].unique()

In [9]:
limited_country_list = ['Italy', 'Korea, South', 'Germany', 'Spain', 'US']

In [10]:
for country in limited_country_list:
    df_plot[country] = np.array(pd_raw[pd_raw['Country/Region'] == country].iloc[:, 4::].sum(axis=0))

In [11]:
df_plot.head()

,date,Italy,"Korea, South",Germany,Spain,US
0,1/22/20,0,1,0,0,1
1,1/23/20,0,1,0,0,1
2,1/24/20,0,2,0,0,2
3,1/25/20,0,2,0,0,2
4,1/26/20,0,3,0,0,5


In [12]:
df_plot.set_index('date').plot()

## Data Type Date

In [13]:
time_index= [datetime.strptime(each, '%m/%d/%y') for each in df_plot.date]
time_string_iso = [each.strftime('%Y-%m-%d') for each in time_index]

In [14]:
df_plot['date'] = time_index

In [15]:
df_plot.head()

,date,Italy,"Korea, South",Germany,Spain,US
0,2020-01-22,0,1,0,0,1
1,2020-01-23,0,1,0,0,1
2,2020-01-24,0,2,0,0,2
3,2020-01-25,0,2,0,0,2
4,2020-01-26,0,3,0,0,5


In [16]:

df_plot.to_csv('../data/processed/COVID_small_flat_table_July 29_.csv', sep = ';', index=False)

  ##  Relational data model

In [17]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)


pd_db = pd_raw.rename(columns= {
    'Country/Region': 'country',
    'Province/State': 'state'
}).drop(['Lat', 'Long'], axis=1)
pd_db  = pd_db.fillna('no')
pd_db.head()

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20
0,no,Afghanistan,0,0,0,0,0,0,0,0,...,38243,38288,38304,38324,38398,38494,38520,38544,38572,38606
1,no,Albania,0,0,0,0,0,0,0,0,...,9728,9844,9967,10102,10255,10406,10553,10704,10860,11021
2,no,Algeria,0,0,0,0,0,0,0,0,...,45158,45469,45773,46071,46364,46653,46938,47216,47488,47752
3,no,Andorra,0,0,0,0,0,0,0,0,...,1199,1199,1215,1215,1215,1261,1261,1301,1301,1344
4,no,Angola,0,0,0,0,0,0,0,0,...,2777,2805,2876,2935,2965,2981,3033,3092,3217,3279


In [18]:
pd_db.set_index(['state', 'country'])

1/22/20  \
state                            country                                     
no                               Afghanistan                             0   
                                 Albania                                 0   
                                 Algeria                                 0   
                                 Andorra                                 0   
                                 Angola                                  0   
                                 Antigua and Barbuda                     0   
                                 Argentina                               0   
                                 Armenia                                 0   
Australian Capital Territory     Australia                               0   
New South Wales                  Australia                               0   
Northern Territory               Australia                               0   
Queensland                       Australia                               0   
South Australia                  Australia                               0   
Tasmania                         Australia                               0   
Victoria                         Australia                               0   
Western Australia                Australia                               0   
no                               Austria                                 0   
                                 Azerbaijan                              0   
                                 Bahamas                                 0   
                                 Bahrain                                 0   
                                 Bangladesh                              0   
                                 Barbados                                0   
                                 Belarus                                 0   
                                 Belgium                                 0   
                                 Belize                                  0   
                                 Benin                                   0   
                                 Bhutan                                  0   
                                 Bolivia                                 0   
                                 Bosnia and Herzegovina                  0   
                                 Botswana                                0   
                                 Brazil                                  0   
                                 Brunei                                  0   
                                 Bulgaria                                0   
                                 Burkina Faso                            0   
                                 Burma                                   0   
                                 Burundi                                 0   
                                 Cabo Verde                              0   
                                 Cambodia                                0   
                                 Cameroon                                0   
Alberta                          Canada                                  0   
British Columbia                 Canada                                  0   
Diamond Princess                 Canada                                  0   
Grand Princess                   Canada                                  0   
Manitoba                         Canada                                  0   
New Brunswick                    Canada                                  0   
Newfoundland and Labrador        Canada                                  0   
Northwest Territories            Canada                                  0   
Nova Scotia                      Canada                                  0   
Ontario                          Canada                                  0   
Prince Edward Island             Canada                                  0   
Quebec     

In [19]:
test_pd = pd_db.set_index(['state', 'country']).T

In [20]:
for each in test_pd.columns:
    print(each)

('no', 'Afghanistan')
('no', 'Albania')
('no', 'Algeria')
('no', 'Andorra')
('no', 'Angola')
('no', 'Antigua and Barbuda')
('no', 'Argentina')
('no', 'Armenia')
('Australian Capital Territory', 'Australia')
('New South Wales', 'Australia')
('Northern Territory', 'Australia')
('Queensland', 'Australia')
('South Australia', 'Australia')
('Tasmania', 'Australia')
('Victoria', 'Australia')
('Western Australia', 'Australia')
('no', 'Austria')
('no', 'Azerbaijan')
('no', 'Bahamas')
('no', 'Bahrain')
('no', 'Bangladesh')
('no', 'Barbados')
('no', 'Belarus')
('no', 'Belgium')
('no', 'Belize')
('no', 'Benin')
('no', 'Bhutan')
('no', 'Bolivia')
('no', 'Bosnia and Herzegovina')
('no', 'Botswana')
('no', 'Brazil')
('no', 'Brunei')
('no', 'Bulgaria')
('no', 'Burkina Faso')
('no', 'Burma')
('no', 'Burundi')
('no', 'Cabo Verde')
('no', 'Cambodia')
('no', 'Cameroon')
('Alberta', 'Canada')
('British Columbia', 'Canada')
('Diamond Princess', 'Canada')
('Grand Princess', 'Canada')
('Manitoba', 'Canada')


In [21]:
test_pd.stack(level =[0,1]).reset_index()


,level_0,state,country,0
0,1/22/20,Alberta,Canada,0.0
1,1/22/20,Anguilla,United Kingdom,0.0
2,1/22/20,Anhui,China,1.0
3,1/22/20,Aruba,Netherlands,0.0
4,1/22/20,Australian Capital Territory,Australia,0.0
...,...,...,...,...
62239,9/11/20,no,West Bank and Gaza,29256.0
62240,9/11/20,no,Western Sahara,10.0
62241,9/11/20,no,Yemen,2007.0
62242,9/11/20,no,Zambia,13323.0


In [22]:
pd_relational_model = pd_db.set_index(['state', 'country']).T.stack(level =[0,1]).reset_index()

pd_relational_model = pd_relational_model.rename(columns ={
    'level_0' : 'date',
    0: 'confirmed'
})

pd_relational_model['date'] = pd.to_datetime(pd_relational_model['date'], format="%m/%d/%y")

In [23]:
pd_relational_model.dtypes

date         datetime64[ns]
state                object
country              object
confirmed           float64
dtype: object

In [24]:
pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv', sep = ';', index=False)

## Group-by apply

In [25]:
pd_JH_data = pd.read_csv('../data/processed/COVID_relational_confirmed.csv', sep = ';', parse_dates = [0])
pd_JH_data = pd_JH_data.sort_values('date').reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


In [26]:
pd_JH_data[pd_JH_data['country'] == 'Germany']

,date,state,country,confirmed
43,2020-01-22,no,Germany,0.0
308,2020-01-23,no,Germany,0.0
575,2020-01-24,no,Germany,0.0
841,2020-01-25,no,Germany,0.0
1107,2020-01-26,no,Germany,0.0
1373,2020-01-27,no,Germany,1.0
1638,2020-01-28,no,Germany,4.0
1905,2020-01-29,no,Germany,4.0
2170,2020-01-30,no,Germany,4.0
2437,2020-01-31,no,Germany,5.0


In [27]:
test_data = pd_JH_data[((pd_JH_data['country'] == 'US') |
                       (pd_JH_data['country'] == 'Germany')) & 
                      (pd_JH_data['date'] >'2020-03-20')]
test_data.head()

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,24508.0
16002,2020-03-22,no,Germany,24873.0
16041,2020-03-22,no,US,33152.0
16269,2020-03-23,no,Germany,29056.0


In [28]:
test_data.groupby(['country']).apply(np.max)

,date,state,country,confirmed
country,,,,
Germany,2020-09-11,no,Germany,259735.0
US,2020-09-11,no,US,6443743.0


In [29]:
reg = linear_model.LinearRegression(fit_intercept=True)


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


In [30]:
def rolling_reg(df_input: pd.DataFrame, col='confirmed', days_back=3):
    result = df_input[col].rolling(
            window=days_back, 
            min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
    return result

In [31]:
pd_DR_result = pd_JH_data[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(rolling_reg, 'confirmed').reset_index()

In [32]:
pd_DR_result = pd_DR_result.rename(columns={
    'confirmed': 'doubling_rate',
    'level_2': 'index'
})


In [33]:
pd_JH_data = pd_JH_data.reset_index()

In [34]:
print(pd_JH_data.columns)

Index(['index', 'date', 'state', 'country', 'confirmed'], dtype='object')


In [35]:
pd_results_large = pd.merge(pd_JH_data, pd_DR_result[['index', 'doubling_rate']], on=['index'], how='left')

In [36]:
pd_results_large[pd_results_large['country'] == 'Germany']

,index,date,state,country,confirmed,doubling_rate
43,43,2020-01-22,no,Germany,0.0,NaN
308,308,2020-01-23,no,Germany,0.0,NaN
575,575,2020-01-24,no,Germany,0.0,NaN
841,841,2020-01-25,no,Germany,0.0,NaN
1107,1107,2020-01-26,no,Germany,0.0,NaN
1373,1373,2020-01-27,no,Germany,1.0,0.666667
1638,1638,2020-01-28,no,Germany,4.0,0.833333
1905,1905,2020-01-29,no,Germany,4.0,2.000000
2170,2170,2020-01-30,no,Germany,4.0,inf
2437,2437,2020-01-31,no,Germany,5.0,8.666667


In [37]:
pd_JH_data[pd_JH_data['country'] == 'Germany']

,index,date,state,country,confirmed
43,43,2020-01-22,no,Germany,0.0
308,308,2020-01-23,no,Germany,0.0
575,575,2020-01-24,no,Germany,0.0
841,841,2020-01-25,no,Germany,0.0
1107,1107,2020-01-26,no,Germany,0.0
1373,1373,2020-01-27,no,Germany,1.0
1638,1638,2020-01-28,no,Germany,4.0
1905,1905,2020-01-29,no,Germany,4.0
2170,2170,2020-01-30,no,Germany,4.0
2437,2437,2020-01-31,no,Germany,5.0


## Filtering the data with groupby apply

In [38]:
5%0

ZeroDivisionError: integer division or modulo by zero